FUCK YOU ALL

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
from sklearn.preprocessing import LabelEncoder
import tensorflow.compat.v1 as tf

# Disable eager execution for Adversarial Debiasing
tf.disable_eager_execution()

# Display output in notebook
%matplotlib inline

# HAROLD CODE

In [ ]:
# Data Visualization

def visualize_biases(data):
    """Generate enhanced visualizations with readable labels and clear category distinctions."""
    # Replace encoded values with readable labels if necessary
    if data['Loan_Status'].dtype in [np.int64, np.float64]:
        data['Loan_Status'] = data['Loan_Status'].replace({1: 'Approved', 0: 'Not Approved'})
    
    if data['Credit_History'].dtype in [np.int64, np.float64]:
        data['Credit_History'] = data['Credit_History'].replace({1: 'Has History', 0: 'No History'})

    plt.figure(figsize=(16, 20))

    # Gender Distribution
    plt.subplot(3, 2, 1)
    plot = sns.countplot(data=data, x='Gender', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Gender', fontsize=14)
    plt.xlabel('Gender', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    # Married Distribution
    plt.subplot(3, 2, 2)
    plot = sns.countplot(data=data, x='Married', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Marital Status', fontsize=14)
    plt.xlabel('Marital Status', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    # Dependents Distribution
    plt.subplot(3, 2, 3)
    plot = sns.countplot(data=data, x='Dependents', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Number of Dependents', fontsize=14)
    plt.xlabel('Number of Dependents', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    # Education Distribution
    plt.subplot(3, 2, 4)
    plot = sns.countplot(data=data, x='Education', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Education Level', fontsize=14)
    plt.xlabel('Education Level', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    # Property Area Distribution
    plt.subplot(3, 2, 5)
    plot = sns.countplot(data=data, x='Property_Area', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Property Area', fontsize=14)
    plt.xlabel('Property Area', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    # Self-Employment Status
    plt.subplot(3, 2, 6)
    plot = sns.countplot(data=data, x='Self_Employed', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Self-Employment Status', fontsize=14)
    plt.xlabel('Self-Employment Status', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    plt.tight_layout()
    plt.show()

    # Numerical Features Analysis
    plt.figure(figsize=(16, 10))

    # Applicant Income
    plt.subplot(2, 2, 1)
    sns.boxplot(data=data, x='Loan_Status', y='ApplicantIncome', palette='coolwarm')
    plt.title('Applicant Income Distribution by Loan Status', fontsize=14)
    plt.xlabel('Loan Status', fontsize=12)
    plt.ylabel('Applicant Income', fontsize=12)

    # Coapplicant Income
    plt.subplot(2, 2, 2)
    sns.boxplot(data=data, x='Loan_Status', y='CoapplicantIncome', palette='coolwarm')
    plt.title('Coapplicant Income Distribution by Loan Status', fontsize=14)
    plt.xlabel('Loan Status', fontsize=12)
    plt.ylabel('Coapplicant Income', fontsize=12)

    # Loan Amount
    plt.subplot(2, 2, 3)
    sns.boxplot(data=data, x='Loan_Status', y='LoanAmount', palette='coolwarm')
    plt.title('Loan Amount Distribution by Loan Status', fontsize=14)
    plt.xlabel('Loan Status', fontsize=12)
    plt.ylabel('Loan Amount', fontsize=12)

    # Credit History
    plt.subplot(2, 2, 4)
    plot = sns.countplot(data=data, x='Credit_History', hue='Loan_Status', palette='coolwarm')
    plt.title('Loan Status by Credit History', fontsize=14)
    plt.xlabel('Credit History', fontsize=12)
    plt.ylabel('Count', fontsize=12)
    plt.legend(title='Loan Status', labels=['Not Approved', 'Approved'], fontsize=10)
    for p in plot.patches:
        plot.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()), 
                      ha='center', va='center', fontsize=10, color='black', 
                      xytext=(0, 5), textcoords='offset points')

    plt.tight_layout()
    plt.show()

# Visualize biases
visualize_biases(data)

# AYMRICK CODE


 *Observation*:
  - Male applicants dominate the dataset.
  - The approval rate for male applicants appears higher than for female applicants.
- *Interpretation*:
  - Gender might play a role in loan approval outcomes, which could indicate bias. Further analysis is needed to determine if this difference is justified by other factors.


In [ ]:
## *1. Loan Status by Gender*
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/content/project-data.csv')


data_info = {
    "Shape": data.shape,
    "Columns": data.columns.tolist(),
    "Data Types": data.dtypes.to_dict(),
    "Missing Values": data.isnull().sum().to_dict(),
    "First Few Rows": data.head().to_dict()
}

data_info

# 1. Clean the data
def clean_data(data):
    """Handle missing values and standardize categorical data."""
    # Fill missing values
    data['Gender'].fillna(data['Gender'].mode()[0], inplace=True)
    data['Married'].fillna(data['Married'].mode()[0], inplace=True)
    data['Dependents'].fillna(data['Dependents'].mode()[0], inplace=True)
    data['Self_Employed'].fillna(data['Self_Employed'].mode()[0], inplace=True)
    data['LoanAmount'].fillna(data['LoanAmount'].median(), inplace=True)
    data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0], inplace=True)
    data['Credit_History'].fillna(data['Credit_History'].mode()[0], inplace=True)

    # Standardize 'Dependents' column
    data['Dependents'] = data['Dependents'].replace({'3+': '3+'})
    return data

# Clean the data
cleaned_data = clean_data(data)
# Display cleaned data info
print("Missing Values After Cleaning:\n", cleaned_data.isnull().sum())

# 2. Preprocess the data
def preprocess_data(data):
    """Encode categorical features and normalize numerical features."""
    # Encode categorical variables
    label_encoders = {}
    categorical_cols = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']
    for col in categorical_cols:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le

    # Normalize numerical features
    scaler = MinMaxScaler()
    numerical_cols = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount']
    data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

    return data, label_encoders

# Preprocess the data
preprocessed_data, label_encoders = preprocess_data(cleaned_data)

# Sensitive attributes for fairness analysis
sensitive_attributes = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Dependents']

# Summary of data preparation
print("Data preparation completed.")
print(f"Sensitive attributes: {sensitive_attributes}")



Missing Values After Cleaning:
 Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


<ipython-input-3-fcada017789e>:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Gender'].fillna(data['Gender'].mode()[0], inplace=True)
<ipython-input-3-fcada017789e>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

# AYMARD CODE

# HERMES CODE

# RAMSES CODE

